In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.linalg as la
np.set_printoptions(precision=2, formatter={'float':lambda x: f"{x:5.02f}"})

In [2]:
import numpy as np
def createBox():
    pts = [
        [0,0,0],
        [0,0,1],
        [0,1,0],
        [1,0,0],
        [1,1,0],
        [0,1,1],
        [1,0,1],
        [1,1,1],
        ]
    return np.array(pts).astype("float32")

box = createBox()
box = box + [50,50,50]
box

array([[50.00, 50.00, 50.00],
       [50.00, 50.00, 51.00],
       [50.00, 51.00, 50.00],
       [51.00, 50.00, 50.00],
       [51.00, 51.00, 50.00],
       [50.00, 51.00, 51.00],
       [51.00, 50.00, 51.00],
       [51.00, 51.00, 51.00]])

In [3]:
# basic pinhole camera
K = [
    [100, 0, 50],
    [0, 100, 50],
    [0,0,1]
    ]
K = np.array(K).astype("float32")

E = [
    [1,0,0,1],
    [0,1,0,1],
    [0,0,1,1],
    [0,0,0,1]
]
E = np.array(E)

E = la.inv(E)[:3,:]

P_native = np.matmul(K,E)
P_native

array([[100.00,  0.00, 50.00, -150.00],
       [ 0.00, 100.00, 50.00, -150.00],
       [ 0.00,  0.00,  1.00, -1.00]])

In [4]:
# view box

box_hat = np.hstack((box,np.ones((box.shape[0],1))))

box_x_hat = np.matmul(P_native,box_hat.T).T
box_x = np.array([box_x_hat[i,:] / box_x_hat[i,-1] for i in range(box_x_hat.shape[0])])[:,:-1]
box_x

array([[150.00, 150.00],
       [148.00, 148.00],
       [150.00, 152.04],
       [152.04, 150.00],
       [152.04, 152.04],
       [148.00, 150.00],
       [150.00, 148.00],
       [150.00, 150.00]])

In [14]:
from src.pycv.perspective import calibrate_dlt, decompose_perspective_projection_matrix

P = calibrate_dlt(box_x, box)

K, R, T = decompose_perspective_projection_matrix(P)
print(f"K\n{K}\nR\n{R}\nT\n{T}")

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00  0.00  0.00]
 [-0.00 -1.00 -0.00]
 [ 0.00 -0.00  1.00]]
T
[ 1.00  1.00  1.00]


In [9]:
box2 = createBox()
box2 = box2 + [5,5,5]
box2

array([[ 5.00,  5.00,  5.00],
       [ 5.00,  5.00,  6.00],
       [ 5.00,  6.00,  5.00],
       [ 6.00,  5.00,  5.00],
       [ 6.00,  6.00,  5.00],
       [ 5.00,  6.00,  6.00],
       [ 6.00,  5.00,  6.00],
       [ 6.00,  6.00,  6.00]])

In [10]:
# view box2, box2_x should be identical to box_x as they are the same camera

box2_x = box_x.copy()
box2_x

array([[150.00, 150.00],
       [148.00, 148.00],
       [150.00, 152.04],
       [152.04, 150.00],
       [152.04, 152.04],
       [148.00, 150.00],
       [150.00, 148.00],
       [150.00, 150.00]])

In [13]:
# calibrate camera using world to image correspondences
P2 = calibrate_dlt(box2_x, box2) # same position, projects to same

K2, R2, T2 = decompose_perspective_projection_matrix(P2)
print(f"K\n{K2}\nR\n{R2}\nT\n{T2}")

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[-44.00 -44.00 -44.00]


In [19]:
from src.pycv.perspective import reprojection_error

repr_err = reprojection_error(P, box_x, box)
repr_err2 = reprojection_error(P2, box2_x, box2)

print(f"original reprojection error \n{repr_err} \n\n estimated reprojection error \n {repr_err2}")


original reprojection error 
[ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00] 

 estimated projection error 
 [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]


estimate transformation using horns method

In [44]:
from src.pycv.perspective import horn_affine_transformation


A = horn_affine_transformation(box,box2)
A = np.vstack((A,[0,0,0,1]))
#P3 = np.matmul(P,la.inv(A)))
P3 = np.matmul(P2,A)

K3, R3, T3 = decompose_perspective_projection_matrix(P3)
print(f"K\n{K3}\nR\n{R3}\nT\n{T3}")
print(f"K\n{K2}\nR\n{R2}\nT\n{T2}")

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[ 1.00  1.00  1.00]
K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[-44.00 -44.00 -44.00]


# TODO test last assumption:
transfer of world points through calibrated camera

In [ ]:
# TODO